In [354]:
import numpy as np
import pandas as pd

hydration_after_p = 0.8
hydration_before_p = 0.045
#Wet Weight(g) after pretreatment
#2% 1h, 12% 1h, 2% 3h 12% 3h
w_60c= np.array([7.8302,4.0316,5.45,4.48]) 
w_80c= np.array([6.4508, 5.181, 5.929, 3.9756]) 
w_100c = np.array([5.6054, 4.1634, 5.9361, 3.9318]) 
w_120c = np.array([4.8918, 3.3319, 5.1648, 2.9348]) 

#Dry Weight(g) after pretreatment
dry_w_60c = w_60c * (1-hydration_after_p)
dry_w_80c = w_80c * (1-hydration_after_p)
dry_w_100c = w_100c * (1-hydration_after_p)
dry_w_120c = w_120c * (1-hydration_after_p)

avg_lost_w_60c = dry_w_60c/2
avg_lost_w_80c = dry_w_80c/2
avg_lost_w_100c = dry_w_100c/2
avg_lost_w_120c = dry_w_120c/2

(array([1.56604, 0.80632, 1.09   , 0.896  ]),
 array([1.29016, 1.0362 , 1.1858 , 0.79512]),
 array([1.12108, 0.83268, 1.18722, 0.78636]),
 array([0.97836, 0.66638, 1.03296, 0.58696]))

In [355]:
def calculate_dry_weight(w, hydration_after_p):
    return w * (1 - hydration_after_p)

def calculate_hplc(hplc, k, m, dilution_factor=5):
    return np.array([dilution_factor*(value-m)/k for value in hplc])

def calculate_mass(c, volume_H):
    return c * volume_H

def calculate_max(values, dry_masses):
    return [values[i]*value for i, value in enumerate(dry_masses)]

def calculate_yield(m, max_m):
    return np.divide(m, max_m)

def calculate_average(pairs):
    return np.mean(pairs, axis=1)

def reshape_into_pairs(array):
    return array.reshape(-1, 2)

def format(input_array):
    multiplied_array = input_array * 100
    str_array = np.array(multiplied_array, dtype=str)
    extracted_array = np.array([x[:4] for x in str_array])
    return extracted_array


In [356]:
def preprocess_data(G_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, dry_masses_before_pretreatment):

    pairs_G_C = reshape_into_pairs(G_samples_hplc)
    pairs_XMG_C = reshape_into_pairs(XMG_samples_hplc)

    avgs_G_C = calculate_average(pairs_G_C)
    avgs_XMG_C = calculate_average(pairs_XMG_C)
  
    c_G = calculate_hplc(avgs_G_C, k=315718, m=4623)
    c_XMG = calculate_hplc(avgs_XMG_C, k=317400, m=10216)
    
    m_G_before = calculate_mass(c_G, volume)
    m_XMG_before = calculate_mass(c_XMG, volume)

    #Kompensera för att vi inte tog allt när vi gjorde hplc
    wet_mass_into_hplc = 2 #g
    moisture = 0.8
    dried_sample = wet_mass_into_hplc * (1-moisture)
    multiply_factor_over_pretreatment = dry_masses_after_p/dried_sample

    m_G = m_G_before * multiply_factor_over_pretreatment
    m_XMG = m_XMG_before * multiply_factor_over_pretreatment

    #Yield
    yield_glucose_procent_process = calculate_yield(m_G, dry_masses_before_pretreatment)
    yield_XMG_procent_process = calculate_yield(m_XMG, dry_masses_before_pretreatment)
    yield_monosacharides_procent_process = np.add(yield_glucose_procent_process, yield_XMG_procent_process)

    #Selectivity
    selectivity_glucose_procent_process = np.divide(m_G, dry_masses_after_p)
    selectivity_XMG_procent_process = np.divide(m_XMG, dry_masses_after_p)
    selectivity_monosacharices_procent_process = np.divide(m_G+m_XMG, dry_masses_after_p)
    

    results = {
        "Dry W Milled(g)": dry_masses_before_pretreatment,
        "Dry W after Pretreatment(g)": dry_masses_after_p,

        "Glucose(g) ": m_G,
        "XMG(g)": m_XMG,
        "Monosacharides(g)": np.add(m_G, m_XMG),

        "Selctivity Glucose (%)": format(selectivity_glucose_procent_process),
        "Selctivity XMG (%)": format(selectivity_XMG_procent_process),
        "Selctivity Monosacharides (%)": format(selectivity_monosacharices_procent_process),

        "Yield Glucose (%)": format(yield_glucose_procent_process),
        "Yield XMG (%)": format(yield_XMG_procent_process),
        "Yield Monosacharides (%)": format(yield_monosacharides_procent_process),
    }

    return results

In [357]:
#Varmast 1h, varmast 3h, kallast 1h, kallast 3h
#Ex 120C 1h, 120C 3h, 60C 1h, 60C 3h

#CHANGE THESE
milled_masses = np.array([2.0924,2.0960,2.0954,2.0985,2.0923,2.0979,2.0962,2.0943])
Glucose_samples_hplc = np.array([590520.1774,  626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333])
XMG_samples_hplc = np.array([149706.8617, 159126.8929,  176542.8571,  168805.7143,  147914.2857,  172857.1429,  146857.1429,  153200.0000])

#Constants
glucose_yields = np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100
xylmangal_values = np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100
volume = 0.02485
hydration_after_p = 0.8

dry_masses_before_p = milled_masses*0.955
average_dry_masses_before_pretreatment = calculate_average(reshape_into_pairs(dry_masses_before_p))
a = average_dry_masses_before_pretreatment

#CHANGE THIS 
dry_masses_after_p = np.array([a[0]*avg_lost_w_120c[0], a[1]*avg_lost_w_120c[2], a[2]*avg_lost_w_60c[0], a[3]*avg_lost_w_60c[2]])


In [358]:
data = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_yields, xylmangal_values, volume, dry_masses_after_p, average_dry_masses_before_pretreatment)


index = ['120C 1h', '120C 3h', '60C 1h', '60C 3h']
process_data_keys = [
    "Dry W Milled(g)",
    "Dry W after Pretreatment(g)",
    "Glucose(g) ",
    "XMG(g)",
    "Monosacharides(g)",
    "Yield Glucose (%)",
    "Yield XMG (%)",
    "Yield Monosacharides (%)",
    "Selctivity Glucose (%)",
    "Selctivity XMG (%)",
    "Selctivity Monosacharides (%)"
]

# Create dataframes
process_data = {key: data[key] for key in process_data_keys}
process_df = pd.DataFrame(process_data)
process_df.index = index

print('2% NaOH batch - Process Data')
print(f"NREL Yield Glucose: {39.54}")
print(f"NREL Yield XMG: {25.24}")
print(f"NREL Yield Monosacharides: {39.54+25.24}")
print()
process_df



dry massses after p: [0.97834092 1.03429626 1.56668247 1.09052524], dry_masses_before_pretreatment: [1.999961   2.00258725 2.0008205  2.00096375]
2% NaOH batch - Process Data
NREL Yield Glucose: 39.54
NREL Yield XMG: 25.24
NREL Yield Monosacharides: 64.78



,Dry W Milled(g),Dry W after Pretreatment(g),Glucose(g),XMG(g),Monosacharides(g),Yield Glucose (%),Yield XMG (%),Yield Monosacharides (%),Selctivity Glucose (%),Selctivity XMG (%),Selctivity Monosacharides (%)
120C 1h,1.999961,0.978341,0.581454,0.138066,0.719520,29.0,6.90,35.9,59.4,14.1,73.5
120C 3h,2.002587,1.034296,0.684820,0.164443,0.849264,34.1,8.21,42.4,66.2,15.8,82.1
60C 1h,2.000820,1.566682,0.959652,0.230246,1.189899,47.9,11.5,59.4,61.2,14.6,75.9
60C 3h,2.000964,1.090525,0.621680,0.149215,0.770895,31.0,7.45,38.5,57.0,13.6,70.6
